In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-adzerrrd
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-adzerrrd
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=dca358923b2589863fa7944b15437f4f0c45051362bdcbb3d24068ec67b1844e
  Stored in directory: /tmp/pip-ephem-wheel-cache-o3hch_kd/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


**Load the Dataset and unzip it**

In [ ]:
!rm -r *
!wget -q https://www.dropbox.com/s/s65hgj63qx77a1i/Lab7.zip
!unzip Lab7.zip
!rm Lab7.zip
!ls

Archive:  Lab7.zip
   creating: Dataset/Test/
   creating: Dataset/Test/0/
  inflating: Dataset/Test/0/input.raw  
  inflating: Dataset/Test/0/myOutput.raw  
  inflating: Dataset/Test/0/output.raw  
   creating: Dataset/Test/1/
  inflating: Dataset/Test/1/input.raw  
 extracting: Dataset/Test/1/myOutput.raw  
 extracting: Dataset/Test/1/output.raw  
   creating: Dataset/Test/10/
  inflating: Dataset/Test/10/input.raw  
  inflating: Dataset/Test/10/myOutput.raw  
  inflating: Dataset/Test/10/output.raw  
   creating: Dataset/Test/2/
  inflating: Dataset/Test/2/input.raw  
 extracting: Dataset/Test/2/myOutput.raw  
 extracting: Dataset/Test/2/output.raw  
   creating: Dataset/Test/3/
  inflating: Dataset/Test/3/input.raw  
 extracting: Dataset/Test/3/myOutput.raw  
 extracting: Dataset/Test/3/output.raw  
   creating: Dataset/Test/4/
  inflating: Dataset/Test/4/input.raw  
 extracting: Dataset/Test/4/myOutput.raw  
 extracting: Dataset/Test/4/output.raw  
   creating: Dataset/Test/5/
  i

# **CUDA Reduction Code Using Simple Algorithm**

Code to be modified where there are TODO indications

In [ ]:
%%writefile lab7-SIMPLE.cu
// Given a list (lst) of length n
// Output its sum = lst[0] + lst[1] + ... + lst[n-1];
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#define BLOCK_SIZE 1024
float* ImportRawFloat(char* filename, int* N)
{
	FILE* handle;
	float* val;
	int i;
	printf("Reading File %s\n", filename);
	if (filename == NULL) {
		return NULL;
	}

	handle = fopen(filename, "r");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return NULL;
	}
	fscanf(handle, "%d", N);
	val = (float*)malloc(*N * sizeof(float));
	for (i = 0; i < *N; i++) {
		fscanf(handle, "%f", val + i);
	}
	fclose(handle);
	return val;

}
int ExportRawFloat(char* filename, float* val, int N)
{
	FILE* handle;
	int i;
	printf("Writing File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}

	handle = fopen(filename, "w");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return 0;
	}
	fprintf(handle, "%d\n", N);
	for (i = 0; i < N; i++) {
		fprintf(handle, "%f\n", val[i]);
	}
	fclose(handle);
	return 1;
}

clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}
__global__ void reduction_simple(float* input, float* output, int len) {
	// TODO: Load a segment of the input vector into shared memory

	// TODO: Traverse the reduction tree

	// TODO: Write the computed sum of the block to the output vector at the
	// correct index
	__shared__ float partialSum[2 * BLOCK_SIZE];
	unsigned int t = threadIdx.x;
	unsigned int start = 2*blockIdx.x*blockDim.x;

	partialSum[t] = input[start + t];
	partialSum[blockDim.x+t] = input[start + blockDim.x+t];

	// Simple Algorithm:
	for (unsigned int stride = 1; stride <= blockDim.x;  stride *= 2) {
   __syncthreads();
   if (t % stride == 0)
     partialSum[2 * t]+= partialSum[2 * t + stride];
 	}

	// TODO: Write the computed sum of the block to the output vector at the
	// correct index

	if (t == 0) {
		output[blockIdx.x] = partialSum[0];
	}
}


int main(int argc, char **argv) {
	int  M;
	float *A; /*input vector A */
	float *C; /*C=reduce(A)*/
	float *D; /*Expected results*/
	float *deviceInput;
	float *deviceOutput;
	int numInputElements;  // number of elements in the input list
	int numOutputElements; // number of elements in the output list
	int i,prt=0;
	float meanDiff = 0.0f;
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[3];

for (ti=0;ti<11;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/Test/%i/input.raw",ti);
	sprintf(argv3,"Dataset/Test/%i/Myoutput.raw",ti);
	sprintf(argv4,"ON");

	printf("Running GPU Reduction Using Simple Algorithm...\n");
	if (strcmp(argv4, "ON") == 0)prt = 1;

  StartingTime = Time_start();
	D = (float*)ImportRawFloat(argv1,&M);
	A = (float*)ImportRawFloat(argv2,&numInputElements);
	printf("The input length is %d\n", numInputElements);
	numOutputElements = numInputElements / (BLOCK_SIZE << 1);
	if (numInputElements % (BLOCK_SIZE << 1)) {
		numOutputElements++;
	}
  printf("The number of output element is %d\n", numOutputElements);
	C = (float*)malloc(numOutputElements * sizeof(float));

  Elapsed_time(StartingTime, "Reading Data.", prt);

	/*TODO: Allocate GPU Memory*/
	StartingTime=Time_start();
	cudaMalloc(&deviceInput, sizeof(float) * numInputElements);
	cudaMalloc(&deviceOutput, sizeof(float) * numOutputElements);
	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
	cudaMemcpy(deviceInput, A, sizeof(float) * numInputElements, cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");

	// TODO: Perform kernel computation here

	dim3 dimGrid(numOutputElements, 1, 1);
	dim3 dimBlock(BLOCK_SIZE, 1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
  reduction_simple << <dimGrid, dimBlock >> >(deviceInput, deviceOutput,numInputElements);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Reduction on Global Memory Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
	cudaMemcpy(C, deviceOutput, sizeof(float) * numOutputElements, cudaMemcpyDeviceToHost);

	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
	cudaFree(deviceInput);
	cudaFree(deviceOutput);
	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);


	/*Perform Success Test*/
/********************************************************************
* Reduce output vector on the host
* NOTE: One could also perform the reduction of the output vector
* recursively and support any size input. For simplicity, we do not
* require that for this lab.
********************************************************************/
	for (i = 1; i < numOutputElements; i++) {
		C[0] += C[i];
	}

		meanDiff = fabs(C[0] - D[0])/(float)numOutputElements;
	if (meanDiff > 0.1) {
		printf("%f Failed\n", meanDiff);

	}
	else {
		printf("%f Passed\n",meanDiff);
	}
	ExportRawFloat(argv3, C, numOutputElements);


	// Free host memory
	free(A);
	free(C);
	free(D);

}

	return 1;
}


Writing lab7-SIMPLE.cu


**Compile GPU Code Using Simple Algorithm**

In [ ]:
!nvcc  lab7-SIMPLE.cu -o lab7-SIMPLE.exe
!ls

Dataset  lab7-SIMPLE.cu  lab7-SIMPLE.exe


**Execute GPU Code Using Simple Algorithm**

In [ ]:
!./lab7-SIMPLE.exe



 +++++++++++++Test 0
Running GPU Reduction Using Simple Algorithm...
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input.raw
The input length is 16
The number of output element is 1
Reading Data. Elapsed Time 0.102000 in mili-seconds
Allocate GPU Memory Elapsed Time 297.964000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.031000 in mili-seconds
Launching kernel
-------->Reduction on Global Memory Kernel Elapsed Time*********** 0.035840 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.028000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.138000 in mili-seconds
0.000000 Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Running GPU Reduction Using Simple Algorithm...
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input.raw
The input length is 64
The number of output element is 1
Reading Data. Elapsed Time 0.078000 in mili-seconds
Allocate GPU Memory Elapsed Time 0.158000 in mili-seconds
C

**Save trace into Lab7SIMPLE.txt**

# **GPU Reduction Code Using Optimized Algorithm**

In [ ]:
%%writefile lab7-OPT.cu
// Given a list (lst) of length n
// Output its sum = lst[0] + lst[1] + ... + lst[n-1];
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#define BLOCK_SIZE 1024
float* ImportRawFloat(char* filename, int* N)
{
	FILE* handle;
	float* val;
	int i;
	printf("Reading File %s\n", filename);
	if (filename == NULL) {
		return NULL;
	}

	handle = fopen(filename, "r");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return NULL;
	}
	fscanf(handle, "%d", N);
	val = (float*)malloc(*N * sizeof(float));
	for (i = 0; i < *N; i++) {
		fscanf(handle, "%f", val + i);
	}
	fclose(handle);
	return val;

}
int ExportRawFloat(char* filename, float* val, int N)
{
	FILE* handle;
	int i;
	printf("Writing File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}

	handle = fopen(filename, "w");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return 0;
	}
	fprintf(handle, "%d\n", N);
	for (i = 0; i < N; i++) {
		fprintf(handle, "%f\n", val[i]);
	}
	fclose(handle);
	return 1;
}

clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}

__global__ void reduction_opt(float* input, float* output, int len) {
	// TODO: Load a segment of the input vector into shared memory

	// TODO: Traverse the reduction tree

	// TODO: Write the computed sum of the block to the output vector at the
	// correct index
	__shared__ volatile float partialSum[2 * BLOCK_SIZE];
	unsigned int t = threadIdx.x, start = 2 * blockIdx.x * BLOCK_SIZE;
	unsigned int i = start + t;

	partialSum[t] = input[i] + input[i+BLOCK_SIZE];
	__syncthreads();

	// TODO: Traverse the reduction tree

	for (unsigned int stride = blockDim.x/2; stride > 32; stride >>= 1) {
		if (t < stride) {
			partialSum[t] += partialSum[t + stride];
		}
		__syncthreads();
	}
	if (t < 32) {
			partialSum[t] += partialSum[t + 32];
			partialSum[t] += partialSum[t + 16];
			partialSum[t] += partialSum[t + 8];
			partialSum[t] += partialSum[t + 4];
			partialSum[t] += partialSum[t + 2];
			partialSum[t] += partialSum[t + 1];
	}

	// TODO: Write the computed sum of the block to the output vector at the
	// correct index
	if (t == 0)
	{
		output[blockIdx.x] = partialSum[0];
	}
}


int main(int argc, char **argv) {
	int  M;
	float *A; /*input vector A */
	float *C; /*C=reduce(A)*/
	float *D; /*Expected results*/
	float *deviceInput;
	float *deviceOutput;
	int numInputElements;  // number of elements in the input list
	int numOutputElements; // number of elements in the output list
	int i,prt=0;
	float meanDiff = 0.0f;
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[3];

for (ti=0;ti<11;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/Test/%i/input.raw",ti);
	sprintf(argv3,"Dataset/Test/%i/Myoutput.raw",ti);
	sprintf(argv4,"ON");

	printf("Running GPU Reduction Using Optimized Algorithm...\n");
	if (strcmp(argv4, "ON") == 0)prt = 1;

  StartingTime = Time_start();
	D = (float*)ImportRawFloat(argv1,&M);
	A = (float*)ImportRawFloat(argv2,&numInputElements);
	printf("The input length is %d\n", numInputElements);
	numOutputElements = numInputElements / (BLOCK_SIZE << 1);
	if (numInputElements % (BLOCK_SIZE << 1)) {
		numOutputElements++;
	}
  printf("The number of output element is %d\n", numOutputElements);
	C = (float*)malloc(numOutputElements * sizeof(float));

  Elapsed_time(StartingTime, "Reading Data.", prt);

	/*TODO: Allocate GPU Memory*/
	StartingTime=Time_start();
	cudaMalloc(&deviceInput, sizeof(float)*numInputElements);
	cudaMalloc(&deviceOutput, sizeof(float)*numOutputElements);

	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
	cudaMemcpy(deviceInput, A, sizeof(float) * numInputElements, cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");

	// TODO: Perform kernel computation here

	dim3 dimGrid(numOutputElements, 1, 1);
	dim3 dimBlock(BLOCK_SIZE, 1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
	reduction_opt << <dimGrid, dimBlock >> >(deviceInput, deviceOutput,numInputElements);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Reduction on Global Memory Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
	cudaMemcpy(C, deviceOutput, sizeof(float) * numOutputElements, cudaMemcpyDeviceToHost);
	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
	cudaFree(deviceInput);
	cudaFree(deviceOutput);

	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);


	/*Perform Success Test*/
/********************************************************************
* Reduce output vector on the host
* NOTE: One could also perform the reduction of the output vector
* recursively and support any size input. For simplicity, we do not
* require that for this lab.
********************************************************************/
	for (i = 1; i < numOutputElements; i++) {
		C[0] += C[i];
	}

		meanDiff = fabs(C[0] - D[0])/(float) numOutputElements;
	if (meanDiff > 0.1) {
		printf("%f Failed\n", meanDiff);

	}
	else {
		printf("%f Passed\n",meanDiff);
	}
	ExportRawFloat(argv3, C, numOutputElements);


	// Free host memory
	free(A);
	free(C);
	free(D);

}

	return 1;
}


Writing lab7-OPT.cu


**Compile Reduction Code Using Optimized Algorithm**

In [ ]:
!nvcc  lab7-OPT.cu -o lab7-OPT.exe
!ls

Dataset  lab7-OPT.cu  lab7-OPT.exe  lab7-SIMPLE.cu  lab7-SIMPLE.exe


**Execute Reduction Code Using Optimized Algorithm**

In [ ]:
!./lab7-OPT.exe



 +++++++++++++Test 0
Running GPU Reduction Using Optimized Algorithm...
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input.raw
The input length is 16
The number of output element is 1
Reading Data. Elapsed Time 0.088000 in mili-seconds
Allocate GPU Memory Elapsed Time 270.242000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.029000 in mili-seconds
Launching kernel
-------->Reduction on Global Memory Kernel Elapsed Time*********** 0.025856 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.022000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.114000 in mili-seconds
0.000000 Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Running GPU Reduction Using Optimized Algorithm...
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input.raw
The input length is 64
The number of output element is 1
Reading Data. Elapsed Time 0.069000 in mili-seconds
Allocate GPU Memory Elapsed Time 0.172000 in mili-sec

Save trace into Lab7OPT.txt